In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import random


def make_noise_img(img: np.ndarray) -> np.ndarray:
    rows, cols, channel = img.shape
    img_noise = img.copy()
    
    for row in range(rows):
        for col in range(cols):
            # Return the next random floating point number in the range [0.0, 1.0)
            seed = random.random()
            if seed < 0.01:
                img_noise[row, col] = np.array([255, 255, 255], dtype="uint8")
    # cv2.imshow("Output", img_noise)    
    # cv2.waitKey(0)
    return img_noise


def average_blur(img_noise):
    start = time.time()
    average_filter = np.ones((3, 3)) / 9
    img_average_blur = cv2.filter2D(img_noise, None, average_filter)
    end = time.time()
    return img_average_blur, round(end-start, 4)


def gaussion_blur(img_noise):
    start = time.time()
    global gaussion_fliter
    gaussion_fliter = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]], dtype="uint8") / 16
    img_gaussion_blur = cv2.filter2D(img_noise, None, gaussion_fliter)
    end = time.time()
    return img_gaussion_blur, round(end-start, 4)


def NL_blur(img_noise):
    start = time.time()
    img_NL_blur = cv2.fastNlMeansDenoisingColored(img_noise, None, 3, 3, 7, 21)
    end = time.time()
    return img_NL_blur, round(end-start, 4)


def HW_1_a():
    plt.figure(figsize=(8, 8))
    show_img = [img_noise, img_average_blur[0], img_gaussion_blur[0], img_NL_blur[0]]
    show_time = ["None", img_average_blur[1], img_gaussion_blur[1], img_NL_blur[1]]
    show_titles = ["Image with noise", "Average blur", "Gaussion blur", "Non-local means blur"]
    for i in range(len(show_titles)):
        plt.subplot(2, 2, 1+i, title=show_titles[i] + " in " + str(show_time[i]) + "s")
        plt.imshow(show_img[i])
    plt.show()


def sobel_detect(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Get x edge
    sobel_x = np.array([
        [1, 2, 1],
        [0, 0, 0],
        [-1, -2, -1]
    ])
    Gx = cv2.filter2D(img, None, sobel_x)

    # Get y edge
    sobel_y = np.array([
        [1, 0, -1],
        [2, 0, -2],
        [1, 0, -1]
    ])
    Gy = cv2.filter2D(img, -1, sobel_y)
    
    Gx = cv2.convertScaleAbs(Gx)
    Gy = cv2.convertScaleAbs(Gy)

    edge = cv2.addWeighted(Gx, 0.5, Gy, 0.5, 0)
    return Gx, Gy, edge


def canny_detect(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gaussion_blur = cv2.filter2D(img, None, gaussion_fliter)
    edge = cv2.Canny(img_gaussion_blur, 50, 150)
    return edge


def HW_1_b():
    plt.figure(figsize=(8, 8))

    show_img = [img_sobel_Gx, img_sobel_Gy, img_sobel_edge, img_canny_edge]
    show_name = ['Gx', 'Gy', '0.5Gx + 0.5Gy', 'Canny']
    for i in range(len(show_img)):
        plt.subplot(2, 2, i + 1, title = show_name[i])
        plt.imshow(show_img[i], cmap = 'gray')
    plt.show()


if __name__ == "__main__":
    # Read the images
    img = cv2.imread("lena.png", flags=cv2.IMREAD_COLOR)
    img = img[:, :, [2, 1, 0]]
    img_paper = cv2.imread("paper.jpg", flags=cv2.IMREAD_COLOR)
    img_papar = img_paper[:, :, [2, 1, 0]]
    
    # Blur images
    img_noise = make_noise_img(img)
    img_average_blur = average_blur(img_noise)
    img_gaussion_blur = gaussion_blur(img_noise)
    img_NL_blur = NL_blur(img_noise)

    # Edge Detect images 
    img_sobel_Gx = sobel_detect(img_paper)[0]
    img_sobel_Gy = sobel_detect(img_paper)[1]
    img_sobel_edge = sobel_detect(img_paper)[2]
    img_canny_edge = canny_detect(img_paper)

    # HW_1_a()
    HW_1_b()
    
